<h1>Importing all the libraries

In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os

client_id = "72c9f2b9d3344c91af50a24f4144228f"
client_secret = "249943a9bbb74b9a8bf04bb7fd93790d"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # sp is the instance of the spotipy api


<h1> Reading data

In [125]:
data = pd.read_excel('./new_data.xlsx')
data


,Artist,title,original_lyrics,hasLyrics,isOrchestral,cleaned_lyrics,lang_detect_spacy,first_musicbrainz_tag,second_musicbrainz_tag,our_genre_tags_based_on_musicmap
0,*NSYNC,Bye Bye Bye,"[Intro: Justin & All]\nHey, hey\nBye bye bye\n...",Yes,No,"Hey, hey Bye bye bye Bye bye! Bye bye! I'm ...",en,pop,boy band,Pop
1,*NSYNC,It‚Äôs Gonna Be Me,[Intro: Justin]\n(It's gonna be me)\nOooh yeah...,Yes,No,"Oooh yeah You might've been hurt, babe That ...",en,pop,boy band,Pop
2,*NSYNC,Tearin‚Äô Up My Heart,[Chorus: JC & Justin]\nIt's tearin' up my hear...,Yes,No,It's tearin' up my heart when I'm with you But...,en,pop,boy band,Pop
3,*NSYNC,Gone,[Verse 1: Justin]\nThere's a thousand words th...,Yes,No,There's a thousand words that I could say To m...,en,pop,boy band,Pop
4,*NSYNC,"Merry Christmas, Happy Holidays","[Intro: Justin, All & JC]\nOooh, ooh ooh\nMerr...",Yes,No,"Oooh, ooh ooh Merry Christmas Happy holidays M...",en,pop,boy band,Pop
...,...,...,...,...,...,...,...,...,...,...
57432,Mr. Gnome,Psychonaut,Oh we fly through the setting sun\nWe climb up...,Yes,No,Oh we fly through the setting sun We climb up ...,en,art rock,alternative rock,Rock
57433,Mr. Gnome,Sit Up & Hum,I saw this man leanin over\nTook his car and d...,Yes,No,I saw this man leanin over Took his car and dr...,en,art rock,alternative rock,Rock
57434,Mr. Gnome,Rise & Shine,Rise and shine\nI‚Äôm coming to you every day ...,Yes,No,Rise and shine I‚Äôm coming to you every day a...,en,art rock,alternative rock,Rock
57435,Mr. Gnome,Be Here Now,No-lyrics-found,No,Unknown,No-lyrics-found,Unknown,art rock,alternative rock,Rock


In [307]:
dataset_artists = list(data['Artist'])
dataset_titles = list(data['title'])
dataset_artists[:5]

['*NSYNC', '*NSYNC', '*NSYNC', '*NSYNC', '*NSYNC']

In [227]:
artist_title_dataset = []
for i in range(len(data)):
    artist_title_dataset.append(str(data['Artist'].iloc[i]) + ' ' + str(data['title'].iloc[i]))
data['Artist+Title'] = artist_title_dataset
data.iloc[0]

Artist                                                                         *NSYNC
title                                                                     Bye Bye Bye
original_lyrics                     [Intro: Justin & All]\nHey, hey\nBye bye bye\n...
hasLyrics                                                                         Yes
isOrchestral                                                                       No
cleaned_lyrics                      Hey, hey Bye bye bye Bye bye! Bye bye!    I'm ...
lang_detect_spacy                                                                  en
first_musicbrainz_tag                                                             pop
second_musicbrainz_tag                                                       boy band
our_genre_tags_based_on_musicmap                                                  Pop
Artist+Title                                                       *NSYNC Bye Bye Bye
Name: 0, dtype: object

In [228]:
artist_title_dataset[0] # this is how we will make a query using the spotifys api


'*NSYNC Bye Bye Bye'

In [229]:
test_result = sp.search(f'{artist_title_dataset[0]}', type='track')
test_result['tracks']['items'][0]['artists'][0]['name'] # extracting the artist name from our 'artist+title' query

'*NSYNC'

In [232]:
test_result['tracks']['items'][0]['artists'][0]['uri']

'spotify:artist:6Ff53KvcvAj5U7Z1vojB5o'

In [211]:
import time
import numpy as np
sleep_min = 2
sleep_max = 3
request_count = 0
# num_artists_available = []
artists_tracks_not_available = []
artists_track_spotify = []
start_time = time.time()
for index in range(len(data[:10])):
    result = sp.search(f'{dataset_artists[index]} {dataset_titles[index]}', type='track')
    if result['tracks']['items']:
        artists_track_spotify.append(f'{dataset_artists[index]} {dataset_titles[index]}')
    else:
        artists_tracks_not_available.append(f'{dataset_artists[index]} {dataset_titles[index]}')
    request_count += 1    
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))



5 artists uris fetched
Loop #: 5
Elapsed Time: 4.7995100021362305 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 8.337558031082153 seconds


In [214]:
artists_tracks_not_available

['*NSYNC It‚Äôs Gonna Be Me',
 '*NSYNC Tearin‚Äô Up My Heart',
 '*shels Butterflies on Luci‚Äôs Way']

In [234]:
def is_english(s):
  return s.isascii()

records_with_only_english_char = []
records_with_foreign_lang_char = []

for items in artist_title_dataset:
  if is_english(items):
    records_with_only_english_char.append(items)
  else:
    records_with_foreign_lang_char.append(items)



In [238]:
print('The total number of records with only english language characters are :- ',len(records_with_only_english_char))
print('Whereas, the total number of records with foreign language characters are :- ',len(records_with_foreign_lang_char))

The total number of records with only english language characters are :-  48891
Whereas, the total number of records with foreign language characters are :-  8546


<h3>We will now check how many out of a sample 100 records which have only english language characters have data available.
Because by our previous observation it was found that all the records with foreign language characters were giving us an empty resposne from the Spotify's API.
</h3>

In [294]:
request_count = 0
start_time = time.time()
artists_titles_present_in_spotify = []
artists_titles_not_present_in_spotify = []
sleep_min = 2
sleep_max = 3
for items in records_with_only_english_char[:100]:
    query  = sp.search(items, type='track')
    if query['tracks']['items']:
        if query['tracks']['items'][0]['preview_url']:
            if query['tracks']['items'][0]['artists'][0]['name'] in items:
                artists_titles_present_in_spotify.append([items,query['tracks']['items'][0]['artists'][0]['name'],query['tracks']['items'][0]['preview_url']])
    else:
        artists_titles_not_present_in_spotify.append(items)
        
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))


5 artists uris fetched
Loop #: 5
Elapsed Time: 3.774610996246338 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 7.4468700885772705 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 10.8352210521698 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 14.390851020812988 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 18.56558609008789 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 22.22084403038025 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 25.948601245880127 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 29.318804025650024 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 32.529646158218384 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 35.93778610229492 seconds
55 artists uris fetched
Loop #: 55
Elapsed Time: 39.619343996047974 seconds
60 artists uris fetched
Loop #: 60
Elapsed Time: 43.6120719909668 seconds
65 artists uris fetched
Loop #: 65
Elapsed Time: 46.69968318939209 seconds
70 artists uris fetched

In [300]:
artists_titles_not_present_in_spotify

['*shels Plains of the Purple Buffalo Pt. 2',
 '*shels Plains of the Purple Buffalo Pt. 1',
 '1 Giant Leap Arrival',
 '1 Giant Leap All Alone (On Eilean Shona)',
 '16 Horsepower Hutterite Mile',
 '16 Horsepower Outlaw Song',
 '2 Chainz 04:00:00']

In [217]:
import unicodedata
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

In [138]:
# artists_with_special_char = []
# alpha_artists = []
# all_artists_dataset = list(data['Artist'])
# for artist in all_artists_dataset:
#     x = str(artist).replace(' ','')
#     if x.isalpha() is False:
#         artists_with_special_char.append(artist)
#     else:
#         alpha_artists.append(artist)
        
# print('The number of artists in our dataset with special characters other than numbers and alphabets are ',len(artists_with_special_char))
# print('The number of artists containing only numbers and aplphabets are  ', len(alpha_artists))


The number of artists in our dataset with special characters other than numbers and alphabets are  4839
The number of artists containing only numbers and aplphabets are   52598


In [139]:
artists_titles_with_special_char = []
alpha_artists_titles = []

for artist in artist_title_dataset:
    x = str(artist).replace(' ','')
    if x.isalpha() is False:
        artists_titles_with_special_char.append(artist)
    else:
        alpha_artists_titles.append(artist)
        
print('The number of artist-title items in our dataset with special characters other than numbers and alphabets are ',len(artists_titles_with_special_char))
print('The number of artist-title items containing only numbers and aplphabets are  ', len(alpha_artists_titles))

The number of artist-title items in our dataset with special characters other than numbers and alphabets are  16414
The number of artist-title items containing only numbers and aplphabets are   41023


In [140]:
alpha_artists_titles

['A Camp Stronger Than Jesus',
 'A Camp I Can Buy You',
 'A Camp Song For The Leftovers',
 'A Camp Algebra',
 'A Camp Charlie Charlie',
 'A Camp The Crowning',
 'A Camp Frequent Flyer',
 'A Camp Hard As A Stone',
 'A Camp Love Has Left The Room',
 'A Camp Golden Teeth And Silver Medals',
 'A Great Big World Say Something',
 'A Great Big World Hold Each Other',
 'A Great Big World I Want a Hippopotamus for Christmas',
 'A Great Big World You',
 'A Great Big World Everyone is Gay',
 'A Great Big World Rockstar',
 'A Great Big World Younger',
 'A Great Big World Already Home',
 'A Great Big World Kaleidoscope',
 'A Guy Called Gerald Voodoo Ray',
 'A Guy Called Gerald Humanity',
 'A Guy Called Gerald Hurry to Go Easy',
 'A Guy Called Gerald The Universe',
 'A Guy Called Gerald Could You Understand',
 'A Guy Called Gerald Multiplies',
 'A Guy Called Gerald Universal Spirit',
 'A Guy Called Gerald Scale Circle',
 'A Guy Called Gerald Glow',
 'A Perfect Circle The Doomed',
 'A Perfect Circle 

In [186]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()
artists_present_in_spotify = []
for index in range(len(alpha_artists_titles[:10])):
    query = sp.search(alpha_artists_titles[index])
    # in the below line of code I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available, and
    # even further I check a third condition whether the artist the we found in our query is the same from our dataset.

    if query['tracks']['items'] and query['tracks']['items'][0]['preview_url'] and query['tracks']['items'][0]['artists'][0]['name'] in alpha_artists_titles[index]:
        artists_present_in_spotify.append([alpha_artists_titles[index],query['tracks']['items'][0]['artists'][0]['name'],query['tracks']['items'][0]['preview_url']])
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 artists uris fetched
Loop #: 5
Elapsed Time: 2.137331962585449 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 5.087433815002441 seconds


In [187]:
# an item of the list below has 3 elements 

'''
1. The query that we made (artist title)
2. The name of the artist from spotify
3. The preview_url of the song that we want
4. The popularity of the track

'''

artists_present_in_spotify[0] 

['A Camp Stronger Than Jesus',
 'A Camp',
 'https://p.scdn.co/mp3-preview/d246ec542b19b5f39498770199410a0389d3c777?cid=72c9f2b9d3344c91af50a24f4144228f']

In [188]:
alpha_artists_titles[:10]

['A Camp Stronger Than Jesus',
 'A Camp I Can Buy You',
 'A Camp Song For The Leftovers',
 'A Camp Algebra',
 'A Camp Charlie Charlie',
 'A Camp The Crowning',
 'A Camp Frequent Flyer',
 'A Camp Hard As A Stone',
 'A Camp Love Has Left The Room',
 'A Camp Golden Teeth And Silver Medals']

In [185]:
query['tracks']['items'][0]['popularity']

3

In [201]:
query['tracks']['items'][0]['uri']

'spotify:track:5inpShAhSWhfYcIFhlSV1F'

In [306]:
x = sp.search('NsY')
x

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=NsY&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6Ff53KvcvAj5U7Z1vojB5o'},
       'href': 'https://api.spotify.com/v1/artists/6Ff53KvcvAj5U7Z1vojB5o',
       'id': '6Ff53KvcvAj5U7Z1vojB5o',
       'name': '*NSYNC',
       'type': 'artist',
       'uri': 'spotify:artist:6Ff53KvcvAj5U7Z1vojB5o'}],
     'available_markets': ['AE',
      'AG',
      'AO',
      'AR',
      'BB',
      'BD',
      'BF',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'DJ',
      'DM',
      'DO',
      'EC',
      'EG',
      'FM',
      'GA',
      'GD',
      'GH',
      'GM',
      'GN',
      'GQ',
      'GT',
      'GW',
      'GY',
     